In [6]:
cd /home/ec2-user/SageMaker/VIP-Dementia

/home/ec2-user/SageMaker/VIP-Dementia


In [7]:
pip install -r requirements.txt

  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached opensmile-2.5.0-py3-none-manylinux_2_17_x86_64.whl.metadata (15 kB)
  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached lightgbm-4.3.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_17_x86_64.whl.metadata (14 kB)
  Using cached pooch-1.8.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached soxr-0.3.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached audobject-0.7.11-py3-none-any.whl.metadata (2.6 kB)
  Using cached audinterface-1.2.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached audeer-2.0.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached audformat-1.1.

In [8]:
cd /home/ec2-user/SageMaker/VIP-Dementia/audio_processing

/home/ec2-user/SageMaker/VIP-Dementia/audio_processing


In [5]:
import numpy as np

# Specify the directory where you want to save the files
HOME_DIRECTORY = os.path.expanduser("~/SageMaker/VIP-Dementia")
save_dir = os.path.join(HOME_DIRECTORY, 'data/processed')

# Function to save datasets
def save_dataset(filename, dataset):
    path = os.path.join(save_dir, filename)
    np.save(path, dataset)
    print(f"Saved {filename} to {save_dir}")

# Saving each dataset
save_dataset('X_train.npy', X_train)
save_dataset('X_valid.npy', X_valid)
save_dataset('X_test.npy', X_test)
save_dataset('y_train.npy', y_train)
save_dataset('y_valid.npy', y_valid)
save_dataset('y_test.npy', y_test)


Saved X_train.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed
Saved X_valid.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed
Saved X_test.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed
Saved y_train.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed
Saved y_valid.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed
Saved y_test.npy to /home/ec2-user/SageMaker/VIP-Dementia/data/processed


In [4]:
import os
import numpy as np

# Specify the directory where you want to save the files
HOME_DIRECTORY = os.path.expanduser("~/SageMaker/VIP-Dementia")
SPLIT_DATA_DIR = os.path.join(HOME_DIRECTORY, 'data/processed')

# Function to load datasets
def load_dataset(filename):
    path = os.path.join(SPLIT_DATA_DIR, filename)
    if os.path.exists(path):
        return np.load(path, allow_pickle=True)
    else:
        print(f"File {filename} not found in {SPLIT_DATA_DIR}. Please check the directory and try again.")
        return None

# Loading each dataset
X_train = load_dataset('X_train.npy')
X_test = load_dataset('X_test.npy')
y_train = load_dataset('y_train.npy')
y_test = load_dataset('y_test.npy')

# Verifying the shapes of the loaded datasets (if they were successfully loaded)
if X_train is not None and y_train is not None:
    print(f"Loaded X_train: {X_train.shape}, y_train: {y_train.shape}")
if X_test is not None and y_test is not None:
    print(f"Loaded X_test: {X_test.shape}, y_test: {y_test.shape}")


Loaded X_train: (183597, 88), y_train: (183597,)
Loaded X_test: (45900, 88), y_test: (45900,)


In [9]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

models = {
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": lgb.LGBMClassifier()
}

Matplotlib is building the font cache; this may take a moment.


In [10]:
model_outputs = {}
for name, model in models.items():
    model.fit(X_train, y_train)  # Train model
    probs = model.predict_proba(X_test)[:, 1]
    preds = model.predict(X_test)
    model_outputs[name] = (probs, preds)
print("All models trained and predictions stored.")

[LightGBM] [Info] Number of positive: 109702, number of negative: 73895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057820 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21030
[LightGBM] [Info] Number of data points in the train set: 183597, number of used features: 88
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.597515 -> initscore=0.395122
[LightGBM] [Info] Start training from score 0.395122
All models trained and predictions stored.


In [11]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

def get_roc_curve(y_test, probs, model_name, color):
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    plt.plot(fpr, tpr, linestyle='-', color=color, label=f'{model_name}', linewidth=0.7)
    plt.xlabel('False Positive Rate', fontsize=8)
    plt.ylabel('True Positive Rate', fontsize=8)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.legend(loc='lower right', fontsize=8)

def get_confusion_matrix(y_test, y_pred, class_names, model_name):
    cm = confusion_matrix(y_test, y_pred)
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  
    
    fig, ax = plt.subplots(figsize=(8, 6)) 
    sns.heatmap(cm_percent, annot=True, fmt=".2%", cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=ax)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

def get_performance_metrics(y_test, model_outputs):
    metrics = {
        'Model': [],
        'Accuracy (%)': [],
        'Precision (%)': [],
        'Recall (%)': [],
        'F1-score (%)': []
    }

    for name, outputs in model_outputs.items():
        y_pred = outputs[1]
        metrics['Model'].append(name)
        metrics['Accuracy (%)'].append(100 * accuracy_score(y_test, y_pred))
        metrics['Precision (%)'].append(100 * precision_score(y_test, y_pred, average="macro"))
        metrics['Recall (%)'].append(100 * recall_score(y_test, y_pred, average="macro"))
        metrics['F1-score (%)'].append(100 * f1_score(y_test, y_pred, average="macro"))

    results_df = pd.DataFrame(metrics)
    return results_df

In [12]:
results_df = get_performance_metrics(y_test, model_outputs)
print(results_df)

           Model  Accuracy (%)  Precision (%)  Recall (%)  F1-score (%)
0  Random Forest     87.612200      87.562190   86.520756     86.953444
1        XGBoost     79.873638      79.296951   78.460295     78.795477
2       LightGBM     76.435730      75.913387   74.321941     74.819715


## RandomForest Analysis

In [11]:
from sklearn.ensemble import RandomForestClassifier

####### Default Random Forest ########
model = RandomForestClassifier(
    random_state=69
)

model.fit(X_train, y_train)

print(f'Default Random Forest Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Default Random Forest Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')


########## Tuned Random Forest #######
model = RandomForestClassifier(
    n_estimators = 500, 
    criterion ='entropy',
    warm_start = True,
    max_features = 'sqrt',
    oob_score = 'True', # more on this below
    random_state=69  
) 

model.fit(X_train, y_train)

print(f'Random Forest Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Random Forest Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

Default Random Forest Model's accuracy on training set is 100.00%
Default Random Forest Model's accuracy on test set is 94.12%



InvalidParameterError: The 'oob_score' parameter of RandomForestClassifier must be an instance of 'bool' or an instance of 'numpy.bool_' or a callable. Got 'True' instead.

## Light GBM Analysis

In [11]:
####### Default LGBM ########
model = lgb.LGBMClassifier(
    verbosity=-1,
    random_state=69
)


model.fit(X_train, y_train)

print(f'Default LGBM\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Default LGBM\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')


########## Tuned LGBM #######

params = {
    'application': 'binary', # for binary classification
#     'num_class' : 1, # used for multi-classes
    'boosting': 'gbdt', # traditional gradient boosting decision tree
    'num_iterations': 100, 
    'learning_rate': 0.05,
    'num_leaves': 62,
    'device': 'cpu', # you can use GPU to achieve faster learning
    'max_depth': -1, # <0 means no limit
    'max_bin': 510, # Small number of bins may reduce training accuracy but can deal with over-fitting
    'lambda_l1': 5, # L1 regularization
    'lambda_l2': 10, # L2 regularization
    'metric' : 'binary_error',
    'subsample_for_bin': 200, # number of samples for constructing bins
    'subsample': 1, # subsample ratio of the training instance
    'colsample_bytree': 0.8, # subsample ratio of columns when constructing the tree
    'min_split_gain': 0.5, # minimum loss reduction required to make further partition on a leaf node of the tree
    'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a leaf
    'min_child_samples': 5# minimum number of data needed in a leaf
}

# Initiate classifier to use
tuned_model = lgb.LGBMClassifier(boosting_type= 'gbdt', 
          objective = 'binary', 
          n_jobs = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'],
          verbosity=-1)

tuned_model.fit(X_train, y_train)

print(f'Tuned LGBM\'s accuracy on training set is {100*tuned_model.score(X_train, y_train):.2f}%')
print(f'Tuned LGBM\'s accuracy on test set is {100*tuned_model.score(X_test, y_test):.2f}%')

Default LGBM's accuracy on training set is 100.00%
Default LGBM's accuracy on test set is 82.35%

Tuned LGBM's accuracy on training set is 100.00%
Tuned LGBM's accuracy on test set is 88.24%


In [12]:
from sklearn.model_selection import GridSearchCV

gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }

grid = GridSearchCV(model, gridParams, verbose=1, cv=4, n_jobs=-1)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

Fitting 4 folds for each of 3456 candidates, totalling 13824 fits
{'boosting_type': 'dart', 'colsample_bytree': 0.65, 'learning_rate': 0.01, 'max_bin': 255, 'n_estimators': 16, 'num_leaves': 6, 'objective': 'binary', 'random_state': 500, 'reg_alpha': 1.2, 'reg_lambda': 1.4, 'subsample': 0.7}
0.5757575757575757


In [27]:
params['colsample_bytree'] = grid.best_params_['colsample_bytree']
params['learning_rate'] = grid.best_params_['learning_rate'] 
params['max_bin'] = grid.best_params_['max_bin']
params['num_leaves'] = grid.best_params_['num_leaves']
params['reg_alpha'] = grid.best_params_['reg_alpha']
params['reg_lambda'] = grid.best_params_['reg_lambda']
params['subsample'] = grid.best_params_['subsample']

params

{'application': 'binary',
 'boosting': 'gbdt',
 'num_iterations': 100,
 'learning_rate': 0.01,
 'num_leaves': 6,
 'device': 'cpu',
 'max_depth': -1,
 'max_bin': 255,
 'lambda_l1': 5,
 'lambda_l2': 10,
 'metric': 'binary_error',
 'subsample_for_bin': 200,
 'subsample': 0.7,
 'colsample_bytree': 0.65,
 'min_split_gain': 0.5,
 'min_child_weight': 1,
 'min_child_samples': 5,
 'reg_alpha': 1.2,
 'reg_lambda': 1.4}

In [29]:
# Initiate classifier to use
grid_model = lgb.LGBMClassifier(boosting_type= 'dart', 
          objective = 'binary', 
          n_jobs = 5, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'],
          verbosity=-1)

grid_model.fit(X_train, y_train)

print(f'Tuned LGBM\'s accuracy on training set is {100*grid_model.score(X_train, y_train):.2f}%')
print(f'Tuned LGBM\'s accuracy on test set is {100*grid_model.score(X_test, y_test):.2f}%')

Tuned LGBM's accuracy on training set is 100.00%
Tuned LGBM's accuracy on test set is 88.24%
